# Optimization Model - Hybrid Energy Generation & Storage: 1 Battery-1 Genset.
=========================================================================================================================
$ 
\\
$

$ 
\\
$

### Python environment.

In [1]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=9, micro=15, releaselevel='final', serial=0)


### Installing PuLP Modeler.

In [2]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.


### Testing PuLP packages.

In [3]:
import pulp
pulp.pulpTestAll()

ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss

Restricted license - for non-production use only - expires 2024-10-28


............................

	 Test that logic put in place for deprecation handling of indexs works
	 Testing 'indexs' param continues to work for LpVariable.dicts
	 Testing 'indexs' param continues to work for LpVariable.matrix
	 Testing 'indices' argument works in LpVariable.dicts
	 Testing 'indices' param continues to work for LpVariable.matrix
	 Testing invalid status
	 Testing continuous LP solution - export dict
	 Testing export dict for LP
	 Testing export dict MIP
	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict
	 Testing continuous LP solution - export solver JSON
	 Testing reading MPS files - binary variable, no constraint names
	 Testing reading MPS files - integer variable
	 Testing reading MPS files - maximize
	 Testing reading MPS files - noname
	 Testing invalid var names
	 Testing logPath argument
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing measuring optimizatio

....

...

..

	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)


.....

	 Testing elastic constraints (penalty unbounded)
	 Testing timeLimit argument


sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
----------------------------------------------------------------------
Ran 1003 tests in 20.520s

OK (skipped=885)


### Importing Libraries.

In [4]:
import numpy as np
from collections import Counter
import math
from pulp import *

### Introducing problem Parameters.

In [12]:
Q_max = 250                                                    # maximal capacity stored on battery
Q_0 = 0.2*Q_max                                                # inital capacity stored on battery
P_A_max = 800                                                  # max output from gen set A
dt = 1                                                         # time step in simulation dt=1min
t_max = 60                                                     # time span for simulation = t_max hours
t = np.atleast_2d(np.arange(0,t_max,dt)).T.conj()
n = len(t)                                                     # number of time steps
m = 1                                                          # number of Gensets used on the ship 
a_j = 1.5                                                      # slope for the linear model of genset j consumption 
fc_j_offset = 500                                              # genset j fuel consumption when no power gnerated (p=0) in g/h 
y_k_j = 1                                                      # genset j pointer at time step k (if genset j is used then = 1 otherwise = 0)
y_k_j_1 = 0                                                    # genset j pointer at time step k-1 (if genset j is used then = 1 otherwise = 0)
K_j_start = dict.fromkeys([x for x in range(60)],0)
K_j_start[0]=30                                                # additional fuel consumption when starting genset j (kg)


# Bringing the Load_window packages onto the path
import sys, os
sys.path.append(os.path.abspath(os.path.join(r'wpath',r'ppath')))

# Now we import modules of Load_window_function
from Load_window import Load_window 


L = Load_window (200,200,10,400,400,10,600,600,10,600,200,30,0,10,20,30)                            # Pre-defined power requirements at each t-value, length(L)=length(t)


# Now we import modules of Dictionaries_add
from Dictionaries_add import merge_dictionaries_1_2 




### Creating the problem variable. 

In [ ]:
Optim = LpProblem('Energy_Opt',LpMinimize)

### Setting-up decision variables.

In [ ]:
V_steps = [x for x in range (60)]

Q_bat = LpVariable.dicts("Q_bat", V_steps, lowBound=0.2*Q_max, upBound=Q_max, cat = LpContinuous)  # Battery charge at time step k.
P_bat = LpVariable.dicts("P_bat", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpContinuous) # Power generated by the battery.
P_A = LpVariable.dicts("P_A", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger)  # Power generated by the Genset j.
P_A_load = LpVariable.dicts("P_A_load", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpInteger) # Power required by the load at time step k.
Z_k_j =  LpVariable.dicts("Z_k_j", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Aditional cost fuel oil consumption when starting Genset j.
SFOC_A = LpVariable.dicts("SFOC_A", V_steps, lowBound=0, upBound=None, cat = LpInteger) # Specific fuel oil consumption.
P_A_bat =  LpVariable.dicts("P_A_bat", V_steps, lowBound=0.2*P_A_max, upBound=0.9*P_A_max, cat = LpContinuous) 
Y_to_bat = LpVariable.dicts("Y_to_bat", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Genset selecter to charge the battery at time step k. 
Y_from_bat = LpVariable.dicts("Y_to_bat", V_steps, lowBound=0, upBound=1, cat = LpBinary) # Battery selecter to transfert to the Genset j st time step k.

### Setting-up the objective function.

In [ ]:
for k in V_steps:
  for j in range(1,m+1):    
    FC_k_j = {k: (P_A[k]*a_j + fc_j_offset * y_k_j)  for k in V_steps}   # linear model for fuel consumption of genset j at time step k
    FC = (sum(FC_k_j[k] * dt/1000 * k for k in V_steps ) ) # sum of the fuel oil comsumption for all gensets over all k steps.
    L_added_cost = sum(Z_k_j[j] * j for j in range(1,m+1)) * sum( K_j_start[k] * k for k in V_steps ) # sum of all of the additional costs including starting costs. 
    Optim += lpSum( FC + L_added_cost ), "objective function minimization fuel oil consumption" 


C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


### Setting-up problem Constraints.

In [ ]:

for k in V_steps: 

  Optim +=  lpSum (merge_dictionaries (P_A_load, P_bat)) == L  # Meet Power Requirement at each time step k
  Optim +=  P_A_load[k] + P_A_bat[k] == P_A[k] #Power Split

  P_A_A = {k: (P_A[k]*(-67/P_A_max) + 260) for k in P_A} 
  Optim += SFOC_A [k] == P_A_A[k]  # Specific Fuel oil consumption


  Q_k =  {k: (P_A_bat[k] - P_bat[k])*dt for k in V_steps} 
  Optim += Q_bat[0] == Q_0 #nitial Charge balance
  Optim += Q_bat[k] + Q_k[k] == Q_bat[k] #Charge balance at time step k" 

  # Genset logical constraints. 
  Optim += P_A[k]  <= (0.9 * P_A_max * y_k_j) # upper bound logical constraint
  Optim += P_A[k]  >=  (0.2 * P_A_max * y_k_j) # lower bound logical constraint


  for j in range(1, m): 
    Optim += lpSum( (Y_to_bat[j] + Y_from_bat[j])  <= 1), #make sure charge and transfer of battery are not runing at the same time"



### Solving the problem.

In [ ]:
status = Optim.solve()

### Optimization result.

In [ ]:
LpStatus[status]

'Optimal'

### Printing constraints per each time step.

In [ ]:
print(Optim.variables)

In [ ]:
for v in Optim.variables():
    print(v.name, "=", v.varValue)

### Calculating the optimized fuel comsumption.

In [ ]:
print("Total fuel comsumption of the trip:", value(Optim.objective),'Kg')

Total fuel comsumption of the trip: 2117.1671642100005 Kg
